In [1]:
#Categorizar una imagen de internet
from PIL import Image
import requests
from io import BytesIO
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

modelo = "DRDENSENET100"

custom_objects = {'KerasLayer': hub.KerasLayer}
modelo = tf.keras.models.load_model(f'./models/DR/{modelo}.h5', custom_objects=custom_objects)
print(modelo)
hermandades = {0:"EL AMOR", 1:"JESUS DESPOJADO", 2:"LA AMARGURA", 3:"LA ESTRELLA", 4:"LA HINIESTA", 5:"LA PAZ", 6:"SAN ROQUE"}

In [2]:
def categorizar(url, hermandades):
  respuesta = requests.get(url)
  img = Image.open(BytesIO(respuesta.content))
  img = np.array(img).astype(float)/255

  img = cv2.resize(img, (224,224))
  prediccion = modelo.predict(img.reshape(-1, 224, 224, 3))
  print(prediccion)
  if max(prediccion[0]) >= 0.7:
      top = np.argmax(prediccion[0], axis=-1)
      return hermandades[top]
  else:
      index = np.argpartition(prediccion[0], -3)[-3:]
      indices = index[np.argsort(-prediccion[0][index])]
      return [hermandades[i] for i in indices]

In [ ]:
def categorizar_local(ruta_imagen, hermandades):
    img = cv2.imread(ruta_imagen)
    
    if img is None:
        raise Exception(f"No se pudo cargar la imagen en {ruta_imagen}")
    img = cv2.resize(img, (224, 224))
    img = img / 255.0
    img = img.reshape(-1, 224, 224, 3)
    prediccion = modelo.predict(img)
    print(prediccion)
    if max(prediccion[0]) >= 0.7:
        top = np.argmax(prediccion[0], axis=-1)
        return hermandades[top]
    else:
        index = np.argpartition(prediccion[0], -3)[-3:]
        indices = index[np.argsort(-prediccion[0][index])]
        return [hermandades[i] for i in indices]

In [3]:
#0 = jesus, 1 = hiniesta, 2 = paz
url = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQjloCNXVbKHmjkDOZiECiazV5J3MpHjxtu5A&usqp=CAU' #debe ser la hiniesta
print(categorizar(url, hermandades))


1/1 [==============================] - 4s 4s/step
[[2.9830700e-05 1.9343544e-04 4.5729589e-04 1.9798514e-05 5.1864310e-05
  2.4805133e-05 9.9915624e-01 3.1826225e-05 3.4938370e-05]]
SAN ROQUE


In [ ]:
ruta_imagen = 'C:\\Users\\super\\OneDrive\\Escritorio\\Captura de pantalla 2024-01-17 184200.png'
print(categorizar_local(ruta_imagen, hermandades))
